--- Day 12: Hill Climbing Algorithm ---

You try contacting the Elves using your handheld device, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where `a` is the lowest elevation, `b` is the next-lowest, and so on up to the highest elevation, `z`.

Also included on the heightmap are marks for your current position (`S`) and the location that should get the best signal (`E`). Your current position (`S`) has elevation `a`, and the location that should get the best signal (`E`) has elevation `z`.

You'd like to reach `E`, but to save energy, you should do it in **as few steps as possible**. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be **at most one higher** than the elevation of your current square; that is, if your current elevation is `m`, you could step to elevation `n`, but not to elevation `o`. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:
```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```
Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the `e` at the bottom. From there, you can spiral around to the goal:
```
v..v<<<<
>v.vv<<^
.>vv>E^^
..v>>>^^
..>>>>>^
```
In the above diagram, the symbols indicate whether the path exits each square moving up (`^`), down (`v`), left (`<`), or right (`>`). The location that should get the best signal is still `E`, and . marks unvisited squares.

This path reaches the goal in **`31`** steps, the fewest possible.

**What is the fewest steps required to move from your current position to the location that should get the best signal?**

In [1]:
# Data Load
example_lines = [
    "Sabqponm",
    "abcryxxl",
    "accszExk",
    "acctuvwj",
    "abdefghi",
]

with open("day_12_input.txt") as _file:
    data_lines = _file.readlines()

In [2]:
# Common Code
from __future__ import annotations

import sys
import typing


class Directions:
    Left = "<"
    Right = ">"
    Up = "^"
    Down = "v"

    @classmethod
    def all(cls):
        return [cls.Left, cls.Right, cls.Up, cls.Down]


class HeightCell:
    _START_SYMBOL = "S"
    _END_SYMBOL = "E"

    def __init__(self, symbol, value):
        self._symbol = symbol
        self._value = value

        self._neighbors: dict[str, typing.Optional[HeightCell]] = dict([direction, None] for direction in Directions.all())

        self.steps_to_end = sys.maxsize
        self.next_step_to_end: typing.Optional[HeightCell] = None

    def __str__(self):
        return self._symbol

    def __repr__(self):
        return str(self)

    @property
    def is_start(self):
        return self._symbol == self._START_SYMBOL

    @property
    def is_end(self):
        return self._symbol == self._END_SYMBOL

    @classmethod
    def parse(cls, symbol):
        value = symbol

        if symbol == cls._START_SYMBOL:
            value = "a"
        elif symbol == cls._END_SYMBOL:
            value = "z"

        return cls(symbol, ord(value))

    def set_neighbor(self, direction, neighbor: HeightCell):
        self._neighbors[direction] = neighbor

    def get_neighbors(self) -> list[HeightCell]:
        return [x for x in self._neighbors.values() if x]

    def can_climb_to(self, other: HeightCell):
        return other._value - self._value <= 1


class Grid:
    def __init__(self, start: HeightCell, end: HeightCell, grid: list[list[HeightCell]]):
        self._start = start
        self._end = end
        self._grid = grid

    def find_path(self):
        def _find_path(target: HeightCell):
            steps_to_end = target.steps_to_end + 1

            neighbors = target.get_neighbors()

            neighbors_that_can_climb = [x for x in neighbors if x.can_climb_to(target)]

            for neighbor in neighbors_that_can_climb:
                if neighbor.is_end:
                    continue

                if not neighbor.next_step_to_end or neighbor.steps_to_end > steps_to_end:
                    neighbor.next_step_to_end = target
                    neighbor.steps_to_end = steps_to_end
                    _find_path(neighbor)

        self._end.steps_to_end = 0
        _find_path(self._end)

    def get_moves_from_start_to_end(self):
        return self._start.steps_to_end

    @classmethod
    def parse(cls, lines):
        end = None
        start = None

        grid: list[list[HeightCell]] = []

        for line in lines:
            grid_line = []
            for symbol in line.strip():
                cell = HeightCell.parse(symbol)
                if cell.is_start:
                    start = cell
                elif cell.is_end:
                    end = cell

                grid_line.append(cell)
            grid.append(grid_line)

        if not start or not end:
            raise RuntimeError("Neither start nor end found")

        height = len(grid)
        width = len(grid[0])

        for j, row in enumerate(grid):
            for i, cell in enumerate(row):
                if j > 0:
                    cell.set_neighbor(Directions.Up, grid[j - 1][i])
                if j < height - 1:
                    cell.set_neighbor(Directions.Down, grid[j + 1][i])
                if i > 0:
                    cell.set_neighbor(Directions.Left, grid[j][i - 1])
                if i < width - 1:
                    cell.set_neighbor(Directions.Right, grid[j][i + 1])

        return cls(start, end, grid)


In [3]:
# Solution 1 Code
def solution_1(lines):
    grid = Grid.parse(lines)
    grid.find_path()

    return grid.get_moves_from_start_to_end()


def test_solution_1():
    actual = solution_1(example_lines)
    expected = 31

    print(actual, expected)
    assert actual == expected
test_solution_1()

31 31


In [4]:
# Solution 1
solution_1(data_lines)

484

_Problem 2_

In [5]:
# Solution 2 Code
def solution_2(lines):
    return ""

def test_solution_2():
    actual = solution_2(example_lines)
    expected = ""
    
    print(actual, expected)
    assert actual == expected
test_solution_2()

In [6]:
# Solution 2
solution_2(data_lines)

''